In [11]:
from pandas import DataFrame
data_structure = DataFrame({ 'Purchases': ['User_id', 'Purchase_id', 'Purchase_datetime', 'Subscription_name', 'Subscription_duration', 'Revenue'],
                                'Installs': ['User_id', 'Media_source', 'Country', 'Install_datetime', 'IDFA', None],
                                'Backend data': ['User_id', 'Device_platform', 'User_birth_date', None, None, None]})
data_structure

,Purchases,Installs,Backend data
0,User_id,User_id,User_id
1,Purchase_id,Media_source,Device_platform
2,Purchase_datetime,Country,User_birth_date
3,Subscription_name,Install_datetime,None
4,Subscription_duration,IDFA,None
5,Revenue,None,None


In [12]:
where: 
● User_id - internal hash id of user
● All fields are in datetime format, i.e. ‘2021-03-01 08:00:00’
● Purchases - one user can have only one purchase
● Subscription_name - name of purchased subscription
● Subscription_duration - duration of subscription in months
● Revenue - LTV of subscription
● Country - install country in ISO format (US, UA, BR etc)
● IDFA - unique device advertising id for preparing look-a-like audience in networks
● Device_platform - iOS, Android
● User_birth_date - users date of birth in format ‘1990-07-25’

Note: 
Installs table includes all users
All purchasers are included in Installs table
All users from Installs table are available in Backend data.


Task: prepare a list of IDFAs, based on which UA team will create a look-a-like audience in Facebook, including the criteria:
Top-5000 purchasers of 6-months subscription by revenue in ascending order
AND who installed an app in the United States of America, Canada, Australia, New Zealand, the United Kingdom (at least in one of the mentioned countries)
AND who
(a) had no activity over that last month (had no events)
OR
(b) had event ‘workout_complete’ at least 5 times over the last 6 months 
AND who have iPhones and who are less than 30yo

Provide SQL query to prepare this list of IDFAs.



SyntaxError: invalid character '●' (U+25CF) (1779140859.py, line 2)

In [ ]:
SELECT 
    IDFA 
FROM 
    Installs 
JOIN 
    Purchases 
USING(User_id)
JOIN 
    Backend_data 
USING(User_id)
--1
WHERE 
    Installs.User_id IN (
                          SELECT 
                            Installs.User_id
                          FROM 
                            Installs 
                          JOIN 
                            Purchases 
                          USING(User_id)
                          WHERE 
                            Subscription_duration = 6 
                          ORDER BY 
                            Revenue
                          LIMIT 5000
                          ) AS task1
--2
AND 
    Country IN ('US', 'CA', 'AU', 'NZ', 'GB') 
--3
AND (
    Installs.User_id IN (
                            SELECT 
                                User_id
                            FROM (
                                  SELECT 
                                         User_id,
                                         Event_datetime,
                                         ROW_NUMBER() OVER (PARTITION BY User_id ORDER BY Event_datetime DESC) AS viev_number
                                  FROM 
                                         App_events
                                 ) AS ranked_events
                            WHERE 
                              viev_number = 1 
                            AND 
                              DATEDIFF(CURRENT_DATE, Event_datetime) < 31
                        )
      OR Installs.User_id IN  (
                                SELECT 
                                  User_id
                                FROM 
                                  App_events
                                WHERE 
                                  DATEDIFF(CURRENT_DATE,Event_datetime) < 183
                                GROUP BY 
                                  User_id
                                HAVING 
                                  COUNT('workout_complete') >=5
                               )
      )
--4
AND 
    Device_platform = "iOS"
AND 
    TIMESTAMPDIFF(YEAR, User_birth_date, CURRENT_DATE) < 30
        